In [1]:
import numpy as np
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField

from openff.interchange import Interchange
from openff.interchange.models import PotentialKey, TopologyKey
from openff.interchange.tests import get_test_file_path

In [2]:
# Load a minimal SMIRNOFF forcefield and Argon topology
argon_ff = ForceField(get_test_file_path("argon.offxml"))

mol = Molecule.from_smiles("[#18]")
mol.generate_conformers(n_conformers=1)

argon_top = Topology.from_molecules(10 * [mol])

In [3]:
# Use a monkey-patched function to parametrize the topology against a force field
off_sys = Interchange.from_smirnoff(force_field=argon_ff, topology=argon_top)

In [4]:
# Look at which ParameterHandler objects from the OpenFF toolkit
# have been made into Potentialhandler objects
off_sys.handlers.keys()

dict_keys(['vdW', 'Electrostatics'])

In [5]:
# Store this handler to inspect its contents
vdw = off_sys.handlers["vdW"]

In [6]:
# Look at some ~metadata
vdw.type, vdw.expression, vdw.independent_variables

('vdW', '4*epsilon*((sigma/r)**12-(sigma/r)**6)', {'r'})

In [7]:
# Return a mapping between atom indices and SMIRKS identifiers
vdw.slot_map

{TopologyKey(atom_indices=(0,), mult=None, bond_order=None): PotentialKey(id='[#18:1]', mult=None, associated_handler='vdW', bond_order=None),
 TopologyKey(atom_indices=(1,), mult=None, bond_order=None): PotentialKey(id='[#18:1]', mult=None, associated_handler='vdW', bond_order=None),
 TopologyKey(atom_indices=(2,), mult=None, bond_order=None): PotentialKey(id='[#18:1]', mult=None, associated_handler='vdW', bond_order=None),
 TopologyKey(atom_indices=(3,), mult=None, bond_order=None): PotentialKey(id='[#18:1]', mult=None, associated_handler='vdW', bond_order=None),
 TopologyKey(atom_indices=(4,), mult=None, bond_order=None): PotentialKey(id='[#18:1]', mult=None, associated_handler='vdW', bond_order=None),
 TopologyKey(atom_indices=(5,), mult=None, bond_order=None): PotentialKey(id='[#18:1]', mult=None, associated_handler='vdW', bond_order=None),
 TopologyKey(atom_indices=(6,), mult=None, bond_order=None): PotentialKey(id='[#18:1]', mult=None, associated_handler='vdW', bond_order=None),

In [8]:
# Return a mapping between SMIRKS identifiers and Potential objects;
# Note the de-duplication, resulting from a many-to-few mapping between
# atoms in the topology and unique parameters in the force field
vdw.potentials

{PotentialKey(id='[#18:1]', mult=None, associated_handler='vdW', bond_order=None): Potential(parameters={'sigma': <Quantity(0.3, 'nanometer')>, 'epsilon': <Quantity(0.1, 'kilojoules_per_mole')>}, map_key=None)}

In [9]:
# Look at this contents of this Potential object
potential_key = PotentialKey(id="[#18:1]", associated_handler="vdW")
vdw.potentials[potential_key]

Potential(parameters={'sigma': <Quantity(0.3, 'nanometer')>, 'epsilon': <Quantity(0.1, 'kilojoules_per_mole')>}, map_key=None)

In [10]:
# Further, look at the particular value of one of its parameters
vdw.potentials[potential_key].parameters["sigma"]

0.3 <Unit('nanometer')>

In [11]:
# Look up, from the highest-level object, this same data, using the
# SMIRKS pattern as a key connecting the topological data to the
# parametrized data
topology_key = TopologyKey(atom_indices=(0,))
off_sys.handlers["vdW"].potentials[
    off_sys.handlers["vdW"].slot_map[topology_key]
].parameters["sigma"]

0.3 <Unit('nanometer')>